In [22]:
import pandas as pd

pd.set_option('display.max_rows', 500)

   ![Stages EDS]('../reports/figures/stages_EDS.png')

# Data Understanding

* RKI, webscrape https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com

## GITHUB csv data

In [16]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd.read_csv(data_path)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/30/22,5/31/22,6/1/22,6/2/22,6/3/22,6/4/22,6/5/22,6/6/22,6/7/22,6/8/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,180259,180347,180419,180520,180584,180615,180615,180688,180741,180784
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,276101,276101,276221,276221,276310,276342,276401,276415,276468,276518
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,265877,265884,265887,265889,265889,265889,265897,265900,265904,265909
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,42894,42894,42894,42894,43067,43067,43067,43067,43067,43224
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761
5,NaN,Antarctica,-71.949900,23.347000,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11
6,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,8163,8253,8253,8295,8295,8378,8378,8378,8378,8378
7,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,9230573,9230573,9230573,9230573,9230573,9230573,9276618,9276618,9276618,9276618
8,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,422963,422963,422963,422963,422963,422963,422963,423006,423006,423006
9,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,131938,132735,133582,134286,134286,134286,134286,136860,137664,138407


## Web Scapping

In [19]:
import requests
from bs4 import BeautifulSoup

In [21]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [23]:
soup = BeautifulSoup(page.content, 'html.parser')

In [25]:
html_table = soup.find('table')

In [27]:
all_rows=html_table.find_all('tr')

In [37]:
final_data_list=[]

In [51]:
for pos,rows in enumerate(all_rows):
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)
 

In [52]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0: 'state',
                                                             1:'cases',
                                                             2:'changes',
                                                             3:'cases_per_100k',
                                                             4:'fatal',
                                                             5:'comment'})

In [53]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,3.714.108,7.826,26.526,"238,9",16.182
3,Bayern,4.956.999,10.994,40.540,"308,5",24.167
4,Berlin,1.061.611,2.237,7.585,"207,0",4.628
5,Branden­burg,797.080,1.290,4.432,"175,1",5.688
6,Bremen,204.310,752,2.545,"374,2",780


## API calls

In [54]:
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronafälle_in_den_Bundesländern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [56]:
import json

In [57]:
json_object=json.loads(data.content)

In [58]:
type(json_object)

dict

In [59]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [62]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])                                           
                                                    
                                                        

In [63]:
pd.DataFrame(full_list)

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2910875,15,760562,1654812000000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,26128.294757,4.573731e+10,2.881496e+06,2563,504.556190,14687,3,"504,6",1
1,2,02,Hamburg,Freie und Hansestadt,1852478,6,594566,1654812000000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,32095.711798,2.089396e+09,4.188002e+05,2679,331.178022,6135,3,"331,2",2
2,3,03,Niedersachsen,Land,8003421,9,2423749,1654812000000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,30283.912342,1.299836e+11,4.008988e+06,9429,454.268244,36357,3,"454,3",3
3,4,04,Bremen,Freie Hansestadt,680130,5,204310,1654812000000,04,4132268b-54de-4327-ac1e-760e915112f1,30039.845324,1.119157e+09,3.357177e+05,780,374.193169,2545,0,"374,2",4
4,5,05,Nordrhein-Westfalen,Land,17925570,10,5384556,1654812000000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,30038.408820,8.782936e+10,2.648673e+06,25504,377.142819,67605,4,"377,1",5
5,6,06,Hessen,Land,6293154,7,1900217,1654812000000,06,93277ac4-e8fc-48c7-8940-028dc2ed66af,30194.986488,5.235913e+10,2.148244e+06,10222,419.026771,26370,4,"419,0",6
6,7,07,Rheinland-Pfalz,Land,4098391,11,1173348,1654812000000,07,e9b4296f-9be2-4e53-9a58-ccf1396cb03d,28629.479227,4.783877e+10,1.774430e+06,5698,325.639989,13346,0,"325,6",7
7,8,08,Baden-Württemberg,Land,11103043,1,3714108,1654812000000,08,80394ddf-c6a4-4a6e-be8e-0259a81b22a9,33451.261965,8.151732e+10,2.544320e+06,16182,238.907478,26526,2,"238,9",8
8,9,09,Bayern,Freistaat,13140183,2,4956999,1654812000000,09,1ff920f4-62cd-4a4f-b8c9-f042f2a3e00a,37723.972337,1.634855e+11,3.898618e+06,24167,308.519295,40540,8,"308,5",9
9,10,10,Saarland,Land,983991,12,319857,1654812000000,10,e3396a6f-8a30-4fdf-8df7-def77dd38bea,32506.089995,6.060692e+09,5.626789e+05,1701,322.665553,3175,0,"322,7",10
